<div style="background-color: #f0efec; color: #1e3a5f; padding: 20px; border-radius: 10px; font-family: sans-serif;">

<div style="text-align: center;">
  <img src="../Images/logo.png" alt="RetroChem Logo" style="max-width: 100%; height: auto;">
</div>

# Introduction 

The following Jupyter notebook briefly presents RetroChem, a pip-installable Python package designed for retrosynthetic analysis. This package was developed to assist chemists and chemical engineers in predicting possible synthetic pathways for target molecules, using a machine learning model trained on the USPTO_50K database. Retrosynthesis is a central concept in organic chemistry, enabling the design of efficient synthetic routes by working backward from the desired product.

This package was created as a collaborative project for the EPFL course Practical Programming in Chemistry. [![GitHub3](https://img.shields.io/badge/EPFL-CH200-red.svg)](https://edu.epfl.ch/studyplan/en/bachelor/chemistry-and-chemical-engineering/coursebook/practical-programming-in-chemistry-CH-200)

Before diving into the code and functionalities of the package, let’s briefly explore the motivations and core concepts that shaped its development.

# How Retrochem came to mind



